In [1]:
import time 
import re
import csv
from getpass import getpass
from time import sleep
from random import randint
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element("xpath",'.//span').text
    try:
        handle = card.find_element("xpath",'.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    
    comment = card.find_element("xpath",'.//div[@data-testid="tweetText"]').text
    responding = card.find_element("xpath",'.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element("xpath",'.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element("xpath",'.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element("xpath",'.//div[@data-testid="like"]').text

    
    tweet = (username, handle, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet  

In [5]:
search_term = input('search term: ')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://twitter.com/")
driver.maximize_window()
sleep(5)

# find search input and search for term
search_input = driver.find_element("xpath",'//input[@aria-label="Search query"]')
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)
sleep(10)

# navigate to historical 'latest' tab
driver.find_element("link text",'Top').click()

search term: ipl


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//input[@aria-label="Search query"]"}
  (Session info: chrome=112.0.5615.139)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x011FDCE3+50899]
	(No symbol) [0x0118E111]
	(No symbol) [0x01095588]
	(No symbol) [0x010C08F9]
	(No symbol) [0x010C0AFB]
	(No symbol) [0x010EF902]
	(No symbol) [0x010DB944]
	(No symbol) [0x010EE01C]
	(No symbol) [0x010DB6F6]
	(No symbol) [0x010B7708]
	(No symbol) [0x010B886D]
	GetHandleVerifier [0x01463EAE+2566302]
	GetHandleVerifier [0x014992B1+2784417]
	GetHandleVerifier [0x0149327C+2759788]
	GetHandleVerifier [0x01295740+672048]
	(No symbol) [0x01198872]
	(No symbol) [0x011941C8]
	(No symbol) [0x011942AB]
	(No symbol) [0x011871B7]
	BaseThreadInitThunk [0x757F7D59+25]
	RtlInitializeExceptionChain [0x7763B74B+107]
	RtlClearBits [0x7763B6CF+191]


In [4]:
# get all tweets on the page
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements("xpath",'//article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(5)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(5) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=112.0.5615.139)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x011FDCE3+50899]
	(No symbol) [0x0118E111]
	(No symbol) [0x01095588]
	(No symbol) [0x0107D333]
	(No symbol) [0x010DF4DB]
	(No symbol) [0x010EDB33]
	(No symbol) [0x010DB6F6]
	(No symbol) [0x010B7708]
	(No symbol) [0x010B886D]
	GetHandleVerifier [0x01463EAE+2566302]
	GetHandleVerifier [0x014992B1+2784417]
	GetHandleVerifier [0x0149327C+2759788]
	GetHandleVerifier [0x01295740+672048]
	(No symbol) [0x01198872]
	(No symbol) [0x011941C8]
	(No symbol) [0x011942AB]
	(No symbol) [0x011871B7]
	BaseThreadInitThunk [0x757F7D59+25]
	RtlInitializeExceptionChain [0x7763B74B+107]
	RtlClearBits [0x7763B6CF+191]


In [5]:
with open('dataset_tweets.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Text', 'Comments', 'Likes', 'Retweets']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

In [7]:
from wordcloud import wordcloud

In [6]:
tweetwd = pd.read_csv('dataset_tweets.csv')
tweetwd

,UserName,Handle,Text,Comments,Likes,Retweets
0,Physics & Astronomy Zone,@zone_astronomy,A view of Earth from the Moon by NASA.A view o...,181.0,"1,273","8,237"
1,NASA,@NASA,One mission. Four astronauts. So many ways to ...,115.0,"1,635","7,934"
2,Troll Football,@TrollFootball,NASA have confirmed that all Man United fans ...,220.0,"2,311",21.5K
3,THE FLAT EARTHER,@TheFlatEartherr,NASA's moon landing hoax was filmed in a studi...,69.0,"1,233","2,768"
4,Wonder of Science,@wonderofscience,The planet Neptune as captured by the Voyager ...,21.0,303,"2,344"
...,...,...,...,...,...,...
148,Matthew Todd,@MrMatthewTodd,NASA https://climate.nasa.gov/evidence/ Oxford...,6.0,1,5
149,Nasa Picture,@NasaPictureBot,149 - M57: The Ring Nebula from Hubble\n \nIt ...,NaN,NaN,2
150,Amisha Mishra,@AmishaMish11196,Indian-origin software and robotics engineer A...,NaN,NaN,1
151,Nature Tv,@NatureTvRw,NASA Astronauts Share Their Space Station Expe...,NaN,3,NaN


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

df = pd.read_csv('dataset_tweets.csv')
text_column = 'Text'
text = ' '.join(df[text_column])

wordcloud = WordCloud(width=800, height=800, background_color='white').generate(text)

plt.figure(figsize=(8, 8))
plt.imshow(wordcloud)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

TypeError: sequence item 68: expected str instance, float found

In [9]:
import pandas as pd

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv("dataset_tweets.csv")

# Drop the columns you want to remove
df = df.drop(columns=["UserName", "Handle","Comments","Likes","Retweets"])

# Write the updated DataFrame to a new CSV file
df.to_csv("updated.csv", index=False)
df

,Text
0,A view of Earth from the Moon by NASA.A view o...
1,One mission. Four astronauts. So many ways to ...
2,NASA have confirmed that all Man United fans ...
3,NASA's moon landing hoax was filmed in a studi...
4,The planet Neptune as captured by the Voyager ...
...,...
148,NASA https://climate.nasa.gov/evidence/ Oxford...
149,149 - M57: The Ring Nebula from Hubble\n \nIt ...
150,Indian-origin software and robotics engineer A...
151,NASA Astronauts Share Their Space Station Expe...


In [10]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = stopwords.words('english')



#actual pre-processing
lemmatizer = WordNetLemmatizer()
for index,row in df.iterrows():
    filter_sentence = []
    sentence = row['Text']
    sentence = re.sub(r'[^\w\s]','',sentence)
    words = nltk.word_tokenize(sentence)
    words = [w for w in words if not w in stop_words]
    for word in words:
        filter_sentence.append(lemmatizer.lemmatize(word))
    print(filter_sentence)
    df.loc[index,'Text'] = filter_sentence


['A', 'view', 'Earth', 'Moon', 'NASAA', 'view', 'Earth', 'Moon', 'NASA']
['One', 'mission', 'Four', 'astronaut', 'So', 'many', 'way', 'watch', 'Artemis', 'II', 'astronaut', 'announcement', 'The', 'thread', 'tell', 'place', 'find', 'historic', 'event', 'We', 'make', 'history', 'Monday', 'April', '3', 'starting', '11am', 'ET', '1500', 'UTCOne', 'mission', 'Four', 'astronaut', 'So', 'many', 'way', 'watch', 'Artemis', 'II', 'astronaut', 'announcement', 'The', 'thread', 'tell', 'place', 'find', 'historic', 'event', 'We', 'make', 'history', 'Monday', 'April', '3', 'starting', '11am', 'ET', '1500', 'UTC']
['NASA', 'confirmed', 'Man', 'United', 'fan', 'made', 'safely', 'back', 'Earth', 'NASA', 'confirmed', 'Man', 'United', 'fan', 'made', 'safely', 'back', 'Earth']
['NASAs', 'moon', 'landing', 'hoax', 'filmed', 'studioNASAs', 'moon', 'landing', 'hoax', 'filmed', 'studio']
['The', 'planet', 'Neptune', 'captured', 'Voyager', '2', 'spacecraft', 'closeup', 'latitudinal', 'cloud', '50', 'km', '30', 

TypeError: expected string or bytes-like object

In [11]:
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [11]:
df

,Text
0,"[A, view, Earth, Moon, NASAA, view, Earth, Moo..."
1,"[One, mission, Four, astronaut, So, many, way,..."
2,"[NASA, confirmed, Man, United, fan, made, safe..."
3,"[NASAs, moon, landing, hoax, filmed, studioNAS..."
4,"[The, planet, Neptune, captured, Voyager, 2, s..."
...,...
148,NASA https://climate.nasa.gov/evidence/ Oxford...
149,149 - M57: The Ring Nebula from Hubble\n \nIt ...
150,Indian-origin software and robotics engineer A...
151,NASA Astronauts Share Their Space Station Expe...


In [12]:
sentences = []

# loop through each row of the data
for index, row in df.iterrows():
    # concatenate the values of each row into a single string
    sentence = ' '.join([str(value) for value in row.values])
    # append the sentence to the list
    sentences.append(sentence)

# combine all sentences into one string, separated by a space
combined_sentence = ' '.join(sentences)

# print or do something with the combined sentence
print(combined_sentence)

['A', 'view', 'Earth', 'Moon', 'NASAA', 'view', 'Earth', 'Moon', 'NASA'] ['One', 'mission', 'Four', 'astronaut', 'So', 'many', 'way', 'watch', 'Artemis', 'II', 'astronaut', 'announcement', 'The', 'thread', 'tell', 'place', 'find', 'historic', 'event', 'We', 'make', 'history', 'Monday', 'April', '3', 'starting', '11am', 'ET', '1500', 'UTCOne', 'mission', 'Four', 'astronaut', 'So', 'many', 'way', 'watch', 'Artemis', 'II', 'astronaut', 'announcement', 'The', 'thread', 'tell', 'place', 'find', 'historic', 'event', 'We', 'make', 'history', 'Monday', 'April', '3', 'starting', '11am', 'ET', '1500', 'UTC'] ['NASA', 'confirmed', 'Man', 'United', 'fan', 'made', 'safely', 'back', 'Earth', 'NASA', 'confirmed', 'Man', 'United', 'fan', 'made', 'safely', 'back', 'Earth'] ['NASAs', 'moon', 'landing', 'hoax', 'filmed', 'studioNASAs', 'moon', 'landing', 'hoax', 'filmed', 'studio'] ['The', 'planet', 'Neptune', 'captured', 'Voyager', '2', 'spacecraft', 'closeup', 'latitudinal', 'cloud', '50', 'km', '30', 

In [9]:
sentences = []

# loop through each row of the data
for index, row in df.iterrows():
    # concatenate the values of each row into a single string
    sentence = ' '.join([str(value) for value in row.values])
    # append the sentence to the list
    sentences.append(sentence)

# combine all sentences into one string, separated by a space
combined_sentence = ' '.join(sentences)

# print or do something with the combined sentence
print(combined_sentence)

['BREAKING', 'Elon', 'Musk', 'file', '60M', 'lawsuit', 'Whoopi', 'Goldberg', 'View', 'Do', 'support', 'thisBREAKING', 'Elon', 'Musk', 'file', '60M', 'lawsuit', 'Whoopi', 'Goldberg', 'View', 'Do', 'support'] ['Joe', 'Biden', 'met', 'privately', 'top', 'executive', 'Facebook', 'Apple', 'Google', 'Including', 'Mark', 'Zuckerberg', 'Tim', 'Cook', 'dozen', 'time', 'Elon', 'Musk', 'however', 'allowed', 'visit', 'White', 'House', 'think', 'himselfJoe', 'Biden', 'met', 'privately', 'top', 'executive', 'Facebook', 'Apple', 'Google', 'Including', 'Mark', 'Zuckerberg', 'Tim', 'Cook', 'dozen', 'time', 'Elon', 'Musk', 'however', 'allowed', 'visit', 'White', 'House', 'think'] ['This', 'wrong', 'elonmusk', 'Reinstate', 'BrandonStrakaThis', 'wrong', 'elonmusk', 'Reinstate', 'BrandonStraka'] ['Hi', 'elonmusk', 'yesterday', 'I', 'posted', 'warning', 'threat', 'terrorist', 'group', 'promising', 'revenge', 'American', 'citizen', 'Twitter', 'forced', 'delete', 'Tweet', 'else', 'would', 'suspend', 'account'

In [36]:
from transformers import pipeline
import os

In [37]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [38]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base")

D:\analytics\downloads\anaconda\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [22]:
for index, row in df.iterrows():
    tweet = row['Text']
    summary = summarizer(tweet, max_length=4, min_length=1, do_sample=True)
    print(summary[0]['summary_text'])

Come and join
The answer is
. 
Fifty-
For more information
FAFul


KeyboardInterrupt: 

In [39]:
summary = summarizer(combined_sentence, max_length=50, min_length=20, do_sample=True)
print(summary[0]['summary_text'])

Token indices sequence length is longer than the specified maximum sequence length for this model (9364 > 512). Running this sequence through the model will result in indexing errors
D:\analytics\downloads\anaconda\lib\site-packages\transformers\generation\utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


'ipl' is a 'game' played in a single match - a match between a team and a player . it's a game of 'italian football',
